# 0 Install and Import

In [1]:
# pip install mediapipe

In [3]:
import mediapipe as mp
import cv2

In [2]:
mp_drawing = mp.solutions.drawing_utils # draw detections from holistic model to opencv
mp_holistic = mp.solutions.holistic # import our holistic model 

# 1 Get Realtime Webcam Feed

In [4]:
frameWidth = 640
frameHeight = 480

cap = cv2.VideoCapture(0)

cap.set(3, frameWidth)
cap.set(4, frameHeight)
cap.set(10, 150)

while cap.isOpened():
    success, img = cap.read()
    if success: 
        cv2.imshow("Raw Input", img)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    else:
        break


In [5]:
cap.release()
cv2.destroyAllWindows()

: 

# 2. Make Detections from Feed

## 2.1 Holistic model:

In [7]:
cap.release()
cv2.destroyAllWindows()

cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        success, image = cap.read()
        
        if success:
            # Recolor Feed (because mediapipe uses RGB while cv2 uses BGR)
            #image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # Make Detections
            results = holistic.process(image)
            # print(results.face_landmarks)  # this will just be numbers/coordinates
            
            # the four landmarks we have:
            # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
            
            # Recolor image back to BGR for rendering, because cv2 loves BGR
            #image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            
            # Draw face landmarks
            mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                                      mp_drawing.DrawingSpec(color=(0,0,255), thickness=1, circle_radius=1),
                                        mp_drawing.DrawingSpec(color=(0,0,245), thickness=1, circle_radius=1)) # FACE_CONNECTIONS: draw the line between the points
            # Right hand
            mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(255,0,0), thickness=2, circle_radius=4),
                                      mp_drawing.DrawingSpec(color=(245,0,0), thickness=2, circle_radius=2))
            # Left Hand
            mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(255,0,0), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,0,0), thickness=2, circle_radius=2)
                                 )
            # Pose Detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(0,245,0), thickness=2, circle_radius=2)
                                 )
                        
        
            cv2.imshow('Result from Holistic Model', image)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        
        else:
            break

## 2.2 For Gesture Recognition

In [27]:
import cv2
import mediapipe as mp

# Function to detect grabbing and putting down actions
def detect_gesture(hand_landmarks):
    # Example logic for detecting grabbing gesture
    thumb_tip = hand_landmarks.landmark[mp.solutions.hands.HandLandmark.THUMB_TIP]
    index_finger_tip = hand_landmarks.landmark[mp.solutions.hands.HandLandmark.INDEX_FINGER_TIP]
    
    distance = cv2.norm((thumb_tip.x, thumb_tip.y), (index_finger_tip.x, index_finger_tip.y))
    
    if distance < 0.1:
        return "Grabbing"
    else:
        return "Putting Down"

# Initialize Mediapipe hands module
mp_hands = mp.solutions.hands.Hands(static_image_mode=False, max_num_hands=2)

# Open video capture
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break
    
    # Flip the frame horizontally for a mirrored view
    frame = cv2.flip(frame, 1)
    
    # Convert the frame to RGB for Mediapipe processing
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Process the frame with Mediapipe hands
    results = mp_hands.process(rgb)
    
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            gesture = detect_gesture(hand_landmarks)
            
            # Draw landmarks and gesture text on the frame
            mp.solutions.drawing_utils.draw_landmarks(
                frame, hand_landmarks, mp.solutions.hands.HAND_CONNECTIONS)
            cv2.putText(frame, gesture, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    # Display the frame
    cv2.imshow('Gesture Recognition', frame)
    
    # Exit if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    

cap.release()
cv2.destroyAllWindows()


2023-07-13 09:18:29.070 Python[46587:5651774] _TIPropertyValueIsValid called with 4 on nil context!
2023-07-13 09:18:29.070 Python[46587:5651774] imkxpc_getApplicationProperty:reply: called with incorrect property value 4, bailing.
2023-07-13 09:18:29.070 Python[46587:5651774] Text input context does not respond to _valueForTIProperty:
2023-07-13 09:18:29.090 Python[46587:5651774] _TIPropertyValueIsValid called with 4 on nil context!
2023-07-13 09:18:29.090 Python[46587:5651774] imkxpc_getApplicationProperty:reply: called with incorrect property value 4, bailing.
2023-07-13 09:18:29.090 Python[46587:5651774] Text input context does not respond to _valueForTIProperty:
2023-07-13 09:18:29.091 Python[46587:5651774] _TIPropertyValueIsValid called with 4 on nil context!
2023-07-13 09:18:29.091 Python[46587:5651774] imkxpc_getApplicationProperty:reply: called with incorrect property value 4, bailing.
2023-07-13 09:18:29.091 Python[46587:5651774] Text input context does not respond to _valueF

## 2.3 Multiple Poses Estimation

Idea:
- Use Mediapipe for single person detection
- Use YOLO to detect person and crop

In [1]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# For webcam input:
cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
  
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # Flip the image horizontally for a later selfie-view display, and convert
    # the BGR image to RGB.
    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    results = pose.process(image)

    # Draw the pose annotation on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    mp_drawing.draw_landmarks(
        image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
    cv2.imshow('MediaPipe Pose', image)
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


### Mediapipe with YOLO Implementation

In [2]:
import torch
import numpy as np
import cv2
import mediapipe as mp


In [5]:
# version 1 (raw input and just object detection)

# Load the model from torch.hub
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')
 
cap = cv2.VideoCapture(0)
while cap.isOpened():
        success, image = cap.read()
        if success:
            # Make Detections
            results = model(image) 
            cv2.imshow('Result',np.squeeze(results.render()))
            if cv2.waitKey(5) & 0xFF == 27:
              break
cap.release()



Using cache found in /Users/julie3399/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-7-17 Python-3.11.1 torch-2.1.0.dev20230328 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [3]:
# version 2 (object detection with pose estimation from Mediapipe)
# doesn't really work, cannot separate the two people: because we need to act directly on the frame, not onto the cropped ones

# Load the model from torch.hub
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')
 
cap = cv2.VideoCapture('Multiple.mp4')
while cap.isOpened():
        success, image = cap.read()
        if success:
            output_frame = image.copy()
            
            # Make Detections
            results = model(image) 
            
            boxes = []
            confidences = []
            class_ids = []
            
            for result in results.xyxy[0]:
                  class_id = result[-1]
                  confidence = result[-2]
                  if confidence > 0.7 and class_id == 0:
                        center_x = int(result[0] * image.shape[1])
                        center_y = int(result[1] * image.shape[0])
                        width = int(result[2] * image.shape[1])
                        height = int(result[3] * image.shape[0])
                        left = int(center_x - width/2)
                        top = int(center_y - height/2)
                        boxes.append([left, top, width, height])
                        confidences.append(float(confidence))
                        class_ids.append(class_id)
            
            pose_tracker = mp.solutions.pose.Pose()
            for i in range(len(boxes)):
                  box = boxes[i]
                  x, y, w, h = box
                  cv2.rectangle(output_frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                  crop = output_frame[y:y+h, x:x+w]
                  
                  # Run pose tracker.
                  crop = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)
                  pose_result = pose_tracker.process(image=crop)
                  pose_landmarks = pose_result.pose_landmarks
                  
                  if pose_landmarks is not None:
                        mp.solutions.drawing_utils.draw_landmarks(output_frame[y:y+h, x:x+w], pose_result.pose_landmarks, mp.solutions.pose.POSE_CONNECTIONS,
                                mp.solutions.drawing_utils.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp.solutions.drawing_utils.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )
            
            cv2.imshow('Result',output_frame)
                  
            if cv2.waitKey(1) & 0xFF == 27:
                  break
cap.release()



Using cache found in /Users/julie3399/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-7-17 Python-3.11.1 torch-2.1.0.dev20230328 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


KeyboardInterrupt: 

In [28]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

image = cv2.imread('temp.jpg')
result = model(image)
result.print()
#result.show()
print('\n', result.xyxy[0]) 

Using cache found in /Users/julie3399/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-7-17 Python-3.11.1 torch-2.1.0.dev20230328 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 
image 1/1: 720x1280 1 person, 1 cell phone
Speed: 1.5ms pre-process, 81.7ms inference, 0.7ms NMS per image at shape (1, 3, 384, 640)



 tensor([[7.19377e+02, 1.66656e+02, 9.88746e+02, 4.22906e+02, 8.74034e-01, 6.70000e+01],
        [2.15004e+02, 1.26850e+02, 1.22454e+03, 7.12456e+02, 6.86175e-01, 0.00000e+00]])


In [14]:
# import the necessary packages
from scipy.spatial import distance as dist
from collections import OrderedDict
import numpy as np
class CentroidTracker:
    def __init__(self, maxDisappeared=50, maxDistance=50):
        # initialize the next unique object ID along with two ordered
        # dictionaries used to keep track of mapping a given object
        # ID to its centroid and number of consecutive frames it has
        # been marked as "disappeared", respectively
        self.nextObjectID = 0
        self.objects = OrderedDict()
        self.disappeared = OrderedDict()

        # store the number of maximum consecutive frames a given
        # object is allowed to be marked as "disappeared" until we
        # need to deregister the object from tracking
        self.maxDisappeared = maxDisappeared

        # store the maximum distance between centroids to associate
        # an object -- if the distance is larger than this maximum
        # distance we'll start to mark the object as "disappeared"
        self.maxDistance = maxDistance

    def register(self, centroid):
        # when registering an object we use the next available object
        # ID to store the centroid
        self.objects[self.nextObjectID] = centroid
        self.disappeared[self.nextObjectID] = 0
        self.nextObjectID += 1

    def deregister(self, objectID):
        # to deregister an object ID we delete the object ID from
        # both of our respective dictionaries
        del self.objects[objectID]
        del self.disappeared[objectID]

    def update(self, rects):
        # check to see if the list of input bounding box rectangles
        # is empty
        if len(rects) == 0:
            # loop over any existing tracked objects and mark them
            # as disappeared
            for objectID in list(self.disappeared.keys()):
                self.disappeared[objectID] += 1

        # if we have reached a maximum number of consecutive
        # frames where a given object has been marked as
        # missing, deregister it
        if self.disappeared[objectID] > self.maxDisappeared:
            self.deregister(objectID)
            
            # return early as there are no centroids or tracking info
            # to update
            return self.objects

        # initialize an array of input centroids for the current frame
        inputCentroids = np.zeros((len(rects), 2), dtype="int")

        # loop over the bounding box rectangles
        for (i, (startX, startY, endX, endY)) in enumerate(rects):
            # use the bounding box coordinates to derive the centroid
            cX = int((startX + endX) / 2.0)
            cY = int((startY + endY) / 2.0)
            inputCentroids[i] = (cX, cY)

        # if we are currently not tracking any objects take the input
        # centroids and register each of them
        if len(self.objects) == 0:
            for i in range(0, len(inputCentroids)):
                self.register(inputCentroids[i])

        # otherwise, are are currently tracking objects so we need to
        # try to match the input centroids to existing object
        # centroids
        else:
            # grab the set of object IDs and corresponding centroids
            objectIDs = list(self.objects.keys())
            objectCentroids = list(self.objects.values())

            # compute the distance between each pair of object
            # centroids and input centroids, respectively -- our
            # goal will be to match an input centroid to an existing
            # object centroid
            D = dist.cdist(np.array(objectCentroids), inputCentroids)

            # in order to perform this matching we must (1) find the
            # smallest value in each row and then (2) sort the row
            # indexes based on their minimum values so that the row
            # with the smallest value as at the *front* of the index
            # list
            rows = D.min(axis=1).argsort()

            # next, we perform a similar process on the columns by
            # finding the smallest value in each column and then
            # sorting using the previously computed row index list
            cols = D.argmin(axis=1)[rows]

            # in order to determine if we need to update, register,
            # or deregister an object we need to keep track of which
            # of the rows and column indexes we have already examined
            usedRows = set()
            usedCols = set()

            # loop over the combination of the (row, column) index
            # tuples
            for (row, col) in zip(rows, cols):
                # if we have already examined either the row or
                # column value before, ignore it
                if row in usedRows or col in usedCols:
                    continue

                # if the distance between centroids is greater than
                # the maximum distance, do not associate the two
                # centroids to the same object
                if D[row, col] > self.maxDistance:
                    continue

                # otherwise, grab the object ID for the current row,
                # set its new centroid, and reset the disappeared
                # counter
                objectID = objectIDs[row]
                self.objects[objectID] = inputCentroids[col]
                self.disappeared[objectID] = 0

                # indicate that we have examined each of the row and
                # column indexes, respectively
                usedRows.add(row)
                usedCols.add(col)

            # compute both the row and column index we have NOT yet
            # examined
            unusedRows = set(range(0, D.shape[0])).difference(usedRows)
            unusedCols = set(range(0, D.shape[1])).difference(usedCols)

            # in the event that the number of object centroids is
            # equal or greater than the number of input centroids
            # we need to check and see if some of these objects have
            # potentially disappeared
            if D.shape[0] >= D.shape[1]:
                # loop over the unused row indexes
                for row in unusedRows:
                    # grab the object ID for the corresponding row
                    # index and increment the disappeared counter
                    objectID = objectIDs[row]
                    self.disappeared[objectID] += 1

                    # check to see if the number of consecutive
                    # frames the object has been marked "disappeared"
                    # for warrants deregistering the object
                    if self.disappeared[objectID] > self.maxDisappeared:
                        self.deregister(objectID)

                    # otherwise, if the number of input centroids is greater
                    # than the number of existing object centroids we need to
                    # register each new input centroid as a trackable object
                    else:
                        for col in unusedCols:
                            self.register(inputCentroids[col])

            # return the set of trackable objects
            return self.objects

In [4]:
# Version 3: Attempt with one algorithm to solve the separation problem

import cv2
import mediapipe as mp

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
# only pose estimation
mp_pose = mp.solutions.pose
# using holistic model
mp_holistic = mp.solutions.holistic

  
pose =  mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.9)

def process_image(image):
    results = pose.process(image)
    mp_drawing.draw_landmarks(
    image,
    results.pose_landmarks,
    mp_pose.POSE_CONNECTIONS,
    landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
    return image

mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)

def process_image_holistic(image):
    results = holistic.process(image)
    
    # Draw face landmarks
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                                      mp_drawing.DrawingSpec(color=(0,0,255), thickness=1, circle_radius=1),
                                        mp_drawing.DrawingSpec(color=(0,0,245), thickness=1, circle_radius=1)) # FACE_CONNECTIONS: draw the line between the points
    # Right hand
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(255,0,0), thickness=2, circle_radius=4),
                                      mp_drawing.DrawingSpec(color=(245,0,0), thickness=2, circle_radius=2))
    # Left Hand
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(255,0,0), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,0,0), thickness=2, circle_radius=2)
                                 )
    # Pose Detections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(0,245,0), thickness=2, circle_radius=2)
                                 )
    
    return image


In [13]:
# Load the YOLOv5 model
yolov5 = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Load the video
# video = cv2.VideoCapture(0)
video = cv2.VideoCapture("Multiple5.mp4")

# Get the video's width, height, and frames per second (fps)
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(video.get(cv2.CAP_PROP_FPS))
# Create a VideoWriter object to save the video
output_file = 'output_video.mp4'  # Specify the output video file name
video_writer = cv2.VideoWriter(output_file, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))



# Process each frame of the video
while True:
  # Read the next frame
  success, frame = video.read()
  if not success:
    break

  # Perform object detection on the frame
  results = yolov5(frame)
  detections = results.xyxy[0]
  

  # Check whether the bounding box centroids are inside the ROI
  for i, detection in enumerate(detections):    
    xmin    = detection[0]
    ymin    = detection[1]
    xmax    = detection[2]
    ymax    = detection[3]
    score   = detection[4]
    class_id= detection[5]
    centroid_x = int(xmin + xmax) // 2
    centroid_y =  int(ymin + ymax) // 2

    #Threshold score
    if score >= 0.7: 
      if class_id == 0:
        color = (255, 0, 0)
        cv2.rectangle(frame, (int(xmin), int(ymin)), (int(xmax), int(ymax)), color, 1)
        cv2.putText(frame, f'Person {i}', (int(xmin), int(ymin)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)

        #padding
        #padding = 25
        person = frame[int(ymin):int(ymax), int(xmin):int(xmax)]

        try:  ### mediapipe
            # process_image(person)
            process_image_holistic(person)
        except:
            pass

      else:
        pass


  # Display the frame
  cv2.imshow("Video", frame)
  video_writer.write(frame)
  if cv2.waitKey(1) & 0xFF == ord('q'):
    break


# Release the video capture object
video.release()
video_writer.release()

cv2.destroyAllWindows()

Using cache found in /Users/julie3399/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-7-17 Python-3.11.1 torch-2.1.0.dev20230328 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [23]:
# Version 4: With tracking in

# Add the Tracker to our Detection part

# Load the YOLOv5 model
yolov5 = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# Load the video
video = cv2.VideoCapture(0)
#video = cv2.VideoCapture("Multiple.mp4")

# Get the video's width, height, and frames per second (fps)
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(video.get(cv2.CAP_PROP_FPS))
# Create a VideoWriter object to save the video
#output_file = 'output_video.mp4'  # Specify the output video file name
#video_writer = cv2.VideoWriter(output_file, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

# Process each frame of the video
while True:
  # Read the next frame
  success, frame = video.read()
  if not success:
    break

  # Perform object detection on the frame
  results = yolov5(frame)
  detections = results.xyxy[0]

  boxes = []
  class_ids = []
  scores = []
  
  for i, detection in enumerate(detections):    
    xmin    = detection[0]
    ymin    = detection[1]
    xmax    = detection[2]
    ymax    = detection[3]
    score   = detection[4]
    class_id = detection[5]
    centroid_x = int(xmin + xmax) // 2
    centroid_y =  int(ymin + ymax) // 2
    
    if score > 0.7 and class_id == 0:
        boxes.append([xmin,ymin,xmax-xmin,ymax-ymin])
        class_ids.append(class_id)
        scores.append(float(score))
        
        ### mediapipe
        person = frame[int(ymin):int(ymax), int(xmin):int(xmax)]
        try:  
            # process_image(person)
            process_image_holistic(person)
        except:
            pass


    # object tracking
    #tracker = EuclideanDistTracker()
    #boxes_ids = tracker.update(boxes)
    for box in boxes:
        x, y, w, h = box
        centroid_x = x + w//2
        centroid_y = y + h//2
        (frame_h, frame_w) = frame.shape[:2]
        
        if centroid_x > frame_w//2:
            id = 0
        else:
            id = 1
        
        cv2.putText(frame, str(id), (int(x), int(y) - 15), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 0), 2)
        cv2.rectangle(frame, (int(x), int(y)), (int(x + w), int(y + h)), (0, 255, 0), 3)

        
  # Display the frame
  cv2.imshow("Video", frame)
  video_writer.write(frame)
  if cv2.waitKey(1) & 0xFF == ord('q'):
    break


# Release the video capture object
video.release()
video_writer.release()

cv2.destroyAllWindows()

Using cache found in /Users/julie3399/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-7-17 Python-3.11.1 torch-2.1.0.dev20230328 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [ ]:
# Euclidean Dist Tracker (Not Used)
import math

# Define the Tracker
class EuclideanDistTracker:
    def __init__(self):
        # Store the center positions of the objects
        self.center_points = {}
        # Keep the count of the IDs
        # each time a new object id detected, the count will increase by one
        self.id_count = 0


    def update(self, objects_rect):
        # Objects boxes and ids
        objects_bbs_ids = []

        # Get center point of new object
        for rect in objects_rect:
            x, y, w, h = rect
            cx = (x + x + w) // 2
            cy = (y + y + h) // 2

            # Find out if that object was detected already
            same_object_detected = False
            for id, pt in self.center_points.items():
                dist = math.hypot(cx - pt[0], cy - pt[1])

                if dist < 25:
                    self.center_points[id] = (cx, cy)
                    print(self.center_points)
                    objects_bbs_ids.append([x, y, w, h, id])
                    same_object_detected = True
                    break

            # New object is detected we assign the ID to that object
            if same_object_detected is False:
                self.center_points[self.id_count] = (cx, cy)
                objects_bbs_ids.append([x, y, w, h, self.id_count])
                self.id_count += 1

        # Clean the dictionary by center points to remove IDS not used anymore
        new_center_points = {}
        for obj_bb_id in objects_bbs_ids:
            _, _, _, _, object_id = obj_bb_id
            center = self.center_points[object_id]
            new_center_points[object_id] = center

        # Update dictionary with IDs not used removed
        self.center_points = new_center_points.copy()
        return objects_bbs_ids
    

In [2]:
# Attempt 0: no progress
def get_multi_poses(coordinates):
   # global variables
   pose_estimator = []
   pose_estimator_dim = []
   # For each object detected
   # WHICH POSE ESTIMATOR TO USE.
   selected_pose_idx=0
                                                      
   if(len(pose_estimator)==0): 
      pose = mp_pose.Pose(min_detection_confidence=0.6,     
               min_tracking_confidence=0.6)
      pose_estimator.append(pose)    
      pose_estimator_dim.append(coordinates)
      selected_pose_idx = len(pose_estimator)-1                            
   elif(<object_id>>len(pose_estimator)):
      thresholdForNew = 100
      prev_high_score = 0
      selected_pose_idx_high =0
      prev_low_score = 1000000000
      selected_pose_idx_low =0
      pose_idx = 0
      for dim in pose_estimator_dim:
         score = compareDist(dim,coordinates)
         if(score > prev_high_score):
            selected_pose_idx_high  =  pose_idx
            prev_high_score = score
         if(score < prev_low_score):                                        
            selected_pose_idx_low  =  pose_idx
            prev_low_score = score
         pose_idx+=1
      if prev_high_score > thresholdForNew:
         pose = mp_pose.Pose(min_detection_confidence=0.6,                   
   min_tracking_confidence=0.6)
         pose_estimator.append(pose)    
         pose_estimator_dim.append(coordinates)
         selected_pose_idx = len(pose_estimator)-1 
      else:
         selected_pose_idx = selected_pose_idx_low
      pose_estimator_dim[selected_pose_idx]=coordinates
                                       
   else:
      pose_idx = 0
      prev_score = 1000000000                                
      for dim in pose_estimator_dim:
         score = compareDist(dim,[x_min, y_min, box_width, box_height])
         if(score < prev_score):                                        
            selected_pose_idx  =  pose_idx
            prev_score = score   
         pose_idx+=1
      pose_estimator_dim[selected_pose_idx]=<detected object's boundary>

SyntaxError: unterminated string literal (detected at line 12) (3999349709.py, line 12)

## 2.4 Object Detection by Motion

In [34]:
import cv2
# from tracker import *

# Create tracker object
#tracker = EuclideanDistTracker()

cap = cv2.VideoCapture(0)

# Object detection from Stable camera
object_detector = cv2.createBackgroundSubtractorMOG2(history=100,varThreshold=16,detectShadows = False)

while True:
    ret, frame = cap.read()
    height, width, _ = frame.shape

    # Extract Region of interest

    # 1. Object Detection
    mask = object_detector.apply(frame)
    _, mask = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        # Calculate area and remove small elements
        area = cv2.contourArea(cnt)
        if area > 3000:
            #cv2.drawContours(roi, [cnt], -1, (0, 255, 0), 2)
            x, y, w, h = cv2.boundingRect(cnt)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 3)

   

    cv2.imshow("roi", frame)
    cv2.imshow("Frame", frame)
    cv2.imshow("Mask", mask)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [5]:
tracker_type = 'MEDIANFLOW'
if tracker_type == 'MEDIANFLOW':
    tracker = cv2.TrackerCSRT_create()

 
    # Read video
    video = cv2.VideoCapture(0)
 
    # Exit if video not opened.
    if not video.isOpened():
        print("Could not open video")
 
    # Read first frame.
    ok, frame = video.read()
    if not ok:
        print('Cannot read video file')
     
    # Define an initial bounding box
    bbox = (287, 23, 86, 320)
 
    # Uncomment the line below to select a different bounding box
    bbox = cv2.selectROI(frame, False)
 
    # Initialize tracker with first frame and bounding box
    ok = tracker.init(frame, bbox)
 
    while True:
        # Read a new frame
        ok, frame = video.read()
        if not ok:
            break
         
        # Start timer
        timer = cv2.getTickCount()
 
        # Update tracker
        ok, bbox = tracker.update(frame)
 
        # Calculate Frames per second (FPS)
        fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer);
 
        # Draw bounding box
        if ok:
            # Tracking success
            p1 = (int(bbox[0]), int(bbox[1]))
            p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
            cv2.rectangle(frame, p1, p2, (255,0,0), 2, 1)
        else :
            # Tracking failure
            cv2.putText(frame, "Tracking failure detected", (100,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)
 
        # Display tracker type on frame
        cv2.putText(frame, tracker_type + " Tracker", (100,20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50),2);
     
        # Display FPS on frame
        cv2.putText(frame, "FPS : " + str(int(fps)), (100,50), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50), 2);
 
        # Display result
        cv2.imshow("Tracking", frame)
 
        # Exit if ESC pressed
        k = cv2.waitKey(1) & 0xff
        if k == 27 : break

cap.release()
cv2.destroyAllWindows()

Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!


2023-07-14 09:44:21.380 Python[56727:6169926] _TIPropertyValueIsValid called with 4 on nil context!
2023-07-14 09:44:21.380 Python[56727:6169926] imkxpc_getApplicationProperty:reply: called with incorrect property value 4, bailing.
2023-07-14 09:44:21.380 Python[56727:6169926] Text input context does not respond to _valueForTIProperty:
2023-07-14 09:44:21.380 Python[56727:6169926] _TIPropertyValueIsValid called with 4 on nil context!
2023-07-14 09:44:21.380 Python[56727:6169926] imkxpc_getApplicationProperty:reply: called with incorrect property value 4, bailing.
2023-07-14 09:44:21.380 Python[56727:6169926] Text input context does not respond to _valueForTIProperty:
2023-07-14 09:44:23.682 Python[56727:6169926] _TIPropertyValueIsValid called with 4 on nil context!
2023-07-14 09:44:23.682 Python[56727:6169926] imkxpc_getApplicationProperty:reply: called with incorrect property value 4, bailing.
2023-07-14 09:44:23.682 Python[56727:6169926] Text input context does not respond to _valueF

KeyboardInterrupt: 

: 

In [25]:
pip install schedule

Note: you may need to restart the kernel to use updated packages.


In [3]:
import cv2
import schedule

cam = cv2.VideoCapture(0)
cv2.namedWindow("Webcam")
img_counter = 0


def capture():
    global img_counter
    img_name = "opencv_frame_{}.png".format(img_counter)
    cv2.imwrite(img_name, frame)
    print("screenshot taken")
    img_counter += 1


# Set up schedule before loop

schedule.every(2).seconds.do(capture)


while True:
    ret, frame = cam.read()

    if not ret:
        print("failed to grab frame")
        break

    cv2.imshow("test", frame)
    schedule.run_pending()

    k = cv2.waitKey(100)  # 1/10 sec delay; no need for separate sleep

    if k % 256 == 27:
        print("closing the app")
        break

cam.release()
cam.destroyAllWindows()

screenshot taken
screenshot taken
screenshot taken
screenshot taken
screenshot taken
screenshot taken
screenshot taken
screenshot taken
screenshot taken
screenshot taken
screenshot taken
screenshot taken
screenshot taken
screenshot taken
screenshot taken
screenshot taken
screenshot taken
screenshot taken
screenshot taken
screenshot taken
screenshot taken
screenshot taken
screenshot taken
screenshot taken
screenshot taken
screenshot taken
screenshot taken
screenshot taken
screenshot taken
closing the app


AttributeError: 'cv2.VideoCapture' object has no attribute 'destroyAllWindows'

: 

In [19]:
!git clone https://github.com/pjreddie/darknet

fatal: 目标路径 'darknet' 已经存在，并且不是一个空目录。


In [20]:
!make

make: *** No targets specified and no makefile found.  Stop.


In [21]:
!wget https://pjreddie.com/media/files/yolov3.weights

--2023-07-17 15:30:13--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: 'yolov3.weights'

yolov3.weights      100%[===================>] 236.52M  10.3MB/s    in 24s     

2023-07-17 15:30:38 (9.88 MB/s) - 'yolov3.weights' saved [248007048/248007048]



In [27]:
class EMADictSmoothing(object):
  """Smoothes pose classification."""

  def __init__(self, window_size=10, alpha=0.2):
    self._window_size = window_size
    self._alpha = alpha

    self._data_in_window = []

  def __call__(self, data):
    """Smoothes given pose classification.

    Smoothing is done by computing Exponential Moving Average for every pose
    class observed in the given time window. Missed pose classes arre replaced
    with 0.
    
    Args:
      data: Dictionary with pose classification. Sample:
          {
            'pushups_down': 8,
            'pushups_up': 2,
          }

    Result:
      Dictionary in the same format but with smoothed and float instead of
      integer values. Sample:
        {
          'pushups_down': 8.3,
          'pushups_up': 1.7,
        }
    """
    # Add new data to the beginning of the window for simpler code.
    self._data_in_window.insert(0, data)
    self._data_in_window = self._data_in_window[:self._window_size]

    # Get all keys.
    keys = set([key for data in self._data_in_window for key, _ in data.items()])

    # Get smoothed values.
    smoothed_data = dict()
    for key in keys:
      factor = 1.0
      top_sum = 0.0
      bottom_sum = 0.0
      for data in self._data_in_window:
        value = data[key] if key in data else 0.0

        top_sum += factor * value
        bottom_sum += factor

        # Update factor.
        factor *= (1.0 - self._alpha)

      smoothed_data[key] = top_sum / bottom_sum

    return smoothed_data

In [28]:
class PoseClassifier(object):
  """Classifies pose landmarks."""

  def __init__(self,
               pose_samples_folder,
               pose_embedder,
               file_extension='csv',
               file_separator=',',
               n_landmarks=33,
               n_dimensions=3,
               top_n_by_max_distance=30,
               top_n_by_mean_distance=10,
               axes_weights=(1., 1., 0.2)):
    self._pose_embedder = pose_embedder
    self._n_landmarks = n_landmarks
    self._n_dimensions = n_dimensions
    self._top_n_by_max_distance = top_n_by_max_distance
    self._top_n_by_mean_distance = top_n_by_mean_distance
    self._axes_weights = axes_weights

    self._pose_samples = self._load_pose_samples(pose_samples_folder,
                                                 file_extension,
                                                 file_separator,
                                                 n_landmarks,
                                                 n_dimensions,
                                                 pose_embedder)

  def _load_pose_samples(self,
                         pose_samples_folder,
                         file_extension,
                         file_separator,
                         n_landmarks,
                         n_dimensions,
                         pose_embedder):
    """Loads pose samples from a given folder.
    
    Required folder structure:
      neutral_standing.csv
      pushups_down.csv
      pushups_up.csv
      squats_down.csv
      ...

    Required CSV structure:
      sample_00001,x1,y1,z1,x2,y2,z2,....
      sample_00002,x1,y1,z1,x2,y2,z2,....
      ...
    """
    # Each file in the folder represents one pose class.
    file_names = [name for name in os.listdir(pose_samples_folder) if name.endswith(file_extension)]

    pose_samples = []
    for file_name in file_names:
      # Use file name as pose class name.
      class_name = file_name[:-(len(file_extension) + 1)]
      
      # Parse CSV.
      with open(os.path.join(pose_samples_folder, file_name)) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=file_separator)
        for row in csv_reader:
          assert len(row) == n_landmarks * n_dimensions + 1, 'Wrong number of values: {}'.format(len(row))
          landmarks = np.array(row[1:], np.float32).reshape([n_landmarks, n_dimensions])
          pose_samples.append(PoseSample(
              name=row[0],
              landmarks=landmarks,
              class_name=class_name,
              embedding=pose_embedder(landmarks),
          ))

    return pose_samples

  def find_pose_sample_outliers(self):
    """Classifies each sample against the entire database."""
    # Find outliers in target poses
    outliers = []
    for sample in self._pose_samples:
      # Find nearest poses for the target one.
      pose_landmarks = sample.landmarks.copy()
      pose_classification = self.__call__(pose_landmarks)
      class_names = [class_name for class_name, count in pose_classification.items() if count == max(pose_classification.values())]

      # Sample is an outlier if nearest poses have different class or more than
      # one pose class is detected as nearest.
      if sample.class_name not in class_names or len(class_names) != 1:
        outliers.append(PoseSampleOutlier(sample, class_names, pose_classification))

    return outliers

  def __call__(self, pose_landmarks):
    """Classifies given pose.

    Classification is done in two stages:
      * First we pick top-N samples by MAX distance. It allows to remove samples
        that are almost the same as given pose, but has few joints bent in the
        other direction.
      * Then we pick top-N samples by MEAN distance. After outliers are removed
        on a previous step, we can pick samples that are closes on average.
    
    Args:
      pose_landmarks: NumPy array with 3D landmarks of shape (N, 3).

    Returns:
      Dictionary with count of nearest pose samples from the database. Sample:
        {
          'pushups_down': 8,
          'pushups_up': 2,
        }
    """
    # Check that provided and target poses have the same shape.
    assert pose_landmarks.shape == (self._n_landmarks, self._n_dimensions), 'Unexpected shape: {}'.format(pose_landmarks.shape)

    # Get given pose embedding.
    pose_embedding = self._pose_embedder(pose_landmarks)
    flipped_pose_embedding = self._pose_embedder(pose_landmarks * np.array([-1, 1, 1]))

    # Filter by max distance.
    #
    # That helps to remove outliers - poses that are almost the same as the
    # given one, but has one joint bent into another direction and actually
    # represnt a different pose class.
    max_dist_heap = []
    for sample_idx, sample in enumerate(self._pose_samples):
      max_dist = min(
          np.max(np.abs(sample.embedding - pose_embedding) * self._axes_weights),
          np.max(np.abs(sample.embedding - flipped_pose_embedding) * self._axes_weights),
      )
      max_dist_heap.append([max_dist, sample_idx])

    max_dist_heap = sorted(max_dist_heap, key=lambda x: x[0])
    max_dist_heap = max_dist_heap[:self._top_n_by_max_distance]

    # Filter by mean distance.
    #
    # After removing outliers we can find the nearest pose by mean distance.
    mean_dist_heap = []
    for _, sample_idx in max_dist_heap:
      sample = self._pose_samples[sample_idx]
      mean_dist = min(
          np.mean(np.abs(sample.embedding - pose_embedding) * self._axes_weights),
          np.mean(np.abs(sample.embedding - flipped_pose_embedding) * self._axes_weights),
      )
      mean_dist_heap.append([mean_dist, sample_idx])

    mean_dist_heap = sorted(mean_dist_heap, key=lambda x: x[0])
    mean_dist_heap = mean_dist_heap[:self._top_n_by_mean_distance]

    # Collect results into map: (class_name -> n_samples)
    class_names = [self._pose_samples[sample_idx].class_name for _, sample_idx in mean_dist_heap]
    result = {class_name: class_names.count(class_name) for class_name in set(class_names)}

    return result

In [26]:
net = cv2.dnn.readNetFromDarknet('yolov3.cfg', 'yolov3.weights')
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA) 
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

In [ ]:
pose = mp.solutions.pose.Pose()
nms_threshold = 0.5
video_cap = cv2.VideoCapture(0)
pose_tracker = mp_pose.Pose(upper_body_only=False)
pose_classification_filter = EMADictSmoothing( window_size=10, alpha=0.2)
pose_samples_folder = '/content/drive/MyDrive/BTP/Database/fitness_poses_csvs_out'
pose_classifier = PoseClassifier( pose_samples_folder=pose_samples_folder, pose_embedder=pose_embedder, top_n_by_max_distance=30, top_n_by_mean_distance=10)
repetition_counter = RepetitionCounter( class_name=class_name, enter_threshold=6, exit_threshold=4)


while True:
    
    success, frame = video_cap.read()
    if not success:
        break

    # Perform object detection using YOLOv3
    blob = cv2.dnn.blobFromImage(frame, 1/255, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    outs = net.forward(net.getUnconnectedOutLayersNames())
    boxes = []
    confidences = []
    class_ids = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence >0.7 and class_id == 0:
                center_x = int(detection[0] * frame.shape[1])
                center_y = int(detection[1] * frame.shape[0])
                width = int(detection[2] * frame.shape[1])
                height = int(detection[3] * frame.shape[0])
                left = int(center_x - width/2)
                top = int(center_y - height/2)
                boxes.append([left, top, width, height])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Apply non-maximum suppression to the boxes
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.96, nms_threshold)    
    # Perform pose detection using Mediapipe
    print(indices)
    for i in indices:
        print(i)

        box = boxes[i[0]]
        x, y, w, h = box
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        crop = frame[y:y+h, x:x+w]

        # Convert the cropped image to RGB and feed it to the pose detection model
        crop_rgb = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)
        crop_rgb.flags.writeable = False
        results = pose.process(crop_rgb)
        crop_rgb.flags.writeable = True
        pose_landmarks = results.pose_landmarks
        crop_rgb = cv2.cvtColor(crop_rgb, cv2.COLOR_RGB2BGR)

        # Draw the pose landmarks on the frame
        mp_drawing.draw_landmarks(crop, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                )
        if pose_landmarks is not None:
            frame_height, frame_width = crop.shape[0], crop.shape[1]
            pose_landmarks = np.array([[lmk.x * frame_width, lmk.y * frame_height, lmk.z * frame_width]
                                    for lmk in pose_landmarks.landmark], dtype=np.float32)
            assert pose_landmarks.shape == (33, 3), 'Unexpected landmarks shape: {}'.format(pose_landmarks.shape)

            # Classify the pose on the current frame.
            pose_classification = pose_classifier(pose_landmarks)

            # Smooth classification using EMA.
            pose_classification_filtered = pose_classification_filter(pose_classification)

            # Count repetitions.
            repetitions_count = repetition_counter(pose_classification_filtered)

        else:
            # No pose => no classification on current frame.
            pose_classification = None

            # Still add empty classification to the filter to maintaing correct
            # smoothing for future frames.
            pose_classification_filtered = pose_classification_filter(dict())
            pose_classification_filtered = None

            # Don't update the counter presuming that person is 'frozen'. Just
            # take the latest repetitions count.
            repetitions_count = repetition_counter.n_repeats


    cv2.putText(frame, str(repetitions_count), 
            (10,60), 
            cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)        

    # Display the resulting frame

    cv2.imshow(frame)

    if cv2.waitKey(1) == ord('q'):
        break

In [1]:
# Run classification on a video(was working with single person video).
import os
import tqdm
import cv2
import numpy as np
import mediapipe as mp
import time

from mediapipe.python.solutions import drawing_utils as mp_drawing

mp_pose = mp.solutions.pose
# Open output video.
video_cap = cv2.VideoCapture(0)

# Load YOLOv3 model
net = cv2.dnn.readNetFromDarknet('yolov3.cfg', 'yolov3.weights')
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

# Initialize variables for graph
frame_count = 0
total_time = 0
processing_times = []

# Define the NMS threshold
nms_threshold = 0.5

frame_idx = 0
output_frame = None

while True:
    # Get next frame of the video.
    success, input_frame = True, cv2.imread('temp.jpg', 0) 
    if not success:
      break
    #starting time for graph
    start_time = time.time()
    
    output_frame = input_frame.copy()
    # Perform object detection using YOLOv3
    blob = cv2.dnn.blobFromImage(input_frame, 1/255, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    outs = net.forward(net.getUnconnectedOutLayersNames())
    boxes = []
    confidences = []
    class_ids = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.7 and class_id == 0:
                center_x = int(detection[0] * input_frame.shape[1])
                center_y = int(detection[1] * input_frame.shape[0])
                width = int(detection[2] * input_frame.shape[1])
                height = int(detection[3] * input_frame.shape[0])
                left = int(center_x - width/2)
                top = int(center_y - height/2)
                boxes.append([left, top, width, height])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    # Apply non-maximum suppression to the boxes
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.96, nms_threshold)    
    
    pose_tracker = mp_pose.Pose()
    #pose_classifier = PoseClassifier(pose_samples_folder=pose_samples_folder,pose_embedder=pose_embedder,top_n_by_max_distance=35,#30top_n_by_mean_distance=10)#10
    
    for i in indices:
        box = boxes[i]
        x, y, w, h = box
        cv2.rectangle(output_frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        crop = output_frame[y:y+h, x:x+w]
        
        # Run pose tracker.
        crop = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)
        result = pose_tracker.process(image=crop)
        pose_landmarks = result.pose_landmarks
        # Draw pose prediction.
        
        if pose_landmarks is not None:
            mp_drawing.draw_landmarks(output_frame[y:y+h, x:x+w], result.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )
        if pose_landmarks is not None:
          # # Get landmarks.
            frame_height, frame_width = crop.shape[0], crop.shape[1]
            pose_landmarks = np.array([[lmk.x * frame_width, lmk.y * frame_height, lmk.z * frame_width]
                                 for lmk in pose_landmarks.landmark], dtype=np.float32)
            assert pose_landmarks.shape == (33, 3), 'Unexpected landmarks shape: {}'.format(pose_landmarks.shape)


        else:
        # No pose => no classification on current frame.
          pose_classification = None

      # Still add empty classification to the filter to maintaing correct
      # smoothing for future frames.
          #pose_classification_filtered = pose_classification_filter(dict())
          pose_classification_filtered = None

    
    #ending time for each frame
    end_time = time.time()
    processing_time = end_time - start_time
    # Accumulate the frame processing time
    total_time += processing_time
    frame_count += 1
     # Store the processing time for plotting
    processing_times.append(processing_time)
    # Create x-axis values for frame numbers
    frame_numbers = np.arange(1, len(processing_times)+1)
    
    # Save the output frame.
    output_frame = cv2.cvtColor(np.array(output_frame), cv2.COLOR_RGB2BGR)

    # Show intermediate frames of the video to track progress.
    cv2.imshow('Output',output_frame)



# Calculate the average frame processing time
average_time = total_time / frame_count
print(f"Average processing time: {average_time:.4f} seconds")


[ WARN:0@6.899] global net_impl.cpp:178 setUpNet DNN module was not built with CUDA backend; switching to CPU
[ERROR:0@6.904] global net_impl.cpp:1169 getLayerShapesRecursively OPENCV/DNN: [Convolution]:(conv_0): getMemoryShapes() throws exception. inputs=1 outputs=0/1 blobs=1
[ERROR:0@6.904] global net_impl.cpp:1172 getLayerShapesRecursively     input[0] = [ 1 1 416 416 ]
[ERROR:0@6.904] global net_impl.cpp:1180 getLayerShapesRecursively     blobs[0] = CV_32FC1 [ 32 3 3 3 ]
[ERROR:0@6.904] global net_impl.cpp:1182 getLayerShapesRecursively Exception message: OpenCV(4.8.0) /Users/xperience/GHA-OpenCV-Python/_work/opencv-python/opencv-python/opencv/modules/dnn/src/layers/convolution_layer.cpp:417: error: (-2:Unspecified error) Number of input channels should be multiple of 3 but got 1 in function 'getMemoryShapes'



error: OpenCV(4.8.0) /Users/xperience/GHA-OpenCV-Python/_work/opencv-python/opencv-python/opencv/modules/dnn/src/layers/convolution_layer.cpp:417: error: (-2:Unspecified error) Number of input channels should be multiple of 3 but got 1 in function 'getMemoryShapes'


In [2]:
video_cap.release()